In [2]:
import math
from scipy.optimize import minimize

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
size = 10  #how many rows from the original data?
card = 5   #cardinality
sigma = np.genfromtxt("orl200-005-a.mat", skip_header=1, dtype='int')
sigma = sigma[0:size,0:size]
mu = pd.read_csv("orl200-005-a.txt",sep=" ", index_col=False, skiprows=1,names=['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-a.rho")
rho = rho.item()
mu = mu.iloc[0:size,0]
mu = np.asarray(mu)
granice = pd.read_csv("orl200-005-a.bds", sep=" ", index_col=False,names=['l_i', 'u_i'])
granice = granice.transpose()
u_i = granice.iloc[1,0:size]
u_i = np.asarray(u_i)

$u_1,\ldots,u_8$ are Lagrangian multipliers 

$u_4,\ldots,u_9$ are arrays of size $size$

$\mathbf x = (x_1,\ldots,x_n,y_1,\ldots,y_n)$ 

In [72]:
2/8

0.25

In [80]:
def objective(x):
    obj = x[0:size].dot(sigma).dot(x[0:size])-(1.0/(k*4))*np.square(u1)-(1.0/(k*4))*np.square(u2)-\
    (1.0/(k*4))*np.square(u3)-(1.0/(k*4))*np.square(u4).sum()-(1.0/(k*4))*np.square(u5).sum()\
    -(1.0/(k*4))*np.square(u6).sum()-(1.0/(k*4))*np.square(u7).sum()-(1.0/(k*4))*np.square(u8).sum()-\
    (1.0/(k*4))*np.square(u9).sum()
    obj += (1.0*k)*max(0, x[0:size].sum() - 1 + (u1/k)/2)**2
    obj += (1.0*k)*max(0, rho-mu.dot(x[0:size])+(u2/k)/2)**2
    obj += (1.0*k)*max(0, size-card-x[size:size*2].sum()+(u3/k)/2)**2
    #constraints for xs
    for i in range(size):
        obj += (1.0*k)*max(0, x[i]-u_i[i]+(u4[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i]+(u5[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i+size]+(u6[i]/k)/2)**2  #y
        obj += (1.0*k)*max(0, x[i+size]-1+(u7[i]/k)/2)**2
        if(x[i]+x[i+size]>=2*t):
            obj += (1.0*k)*max(0, (x[i]-t)*(x[i+size]-t)+(u8[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((x[i]-t)**2+(x[i+size]-t)**2)+(u8[i]/k)/2)**2
        if(-x[i]+x[i+size]>=2*t):
            obj += (1.0*k)*max(0, (-x[i]-t)*(x[i+size]-t)+(u9[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((-x[i]-t)**2+(x[i+size]-t)**2)+(u9[i]/k)/2)**2
    return obj

In [81]:
def phi(a, b, t):
    if(a+b>=2*t):
        result = (a-t)*(b-t)
    else:
        result = -1/2*((a-t)**2+(b-t)**2)
    return result

def phi_(a, b, t):
    if(-a+b >= 2*t):
        result = (-a-t)*(b-t)
    else:
        result = -1/2*((-a-t)**2+(b-t)**2)
    return result

In [ ]:
%%time
t = 1
k = 1/t   #penalty parameter
u1 = 0
u2 = 0
u3 = 0
u4 = np.zeros(size)
u5 = np.zeros(size)
u6 = np.zeros(size)
u7 = np.zeros(size)
u8 = np.zeros(size)
u9 = np.zeros(size)
viol = math.inf     #at the beggining violence is infinity
x0 = np.append(np.zeros(size), np.ones(size))  #starting point
old_viol = viol
eps = 10**-7 #tolerance level
cond = 1
step = 0.9
it = 0
while ((eps <= viol) & (10**-10 <= t)):
    it += 1
    #inner loop
    x = minimize(objective, x0).x
    violence = np.zeros(3+5*size)
    violence[0] = max(0,x[0:size].sum()-1)
    violence[1] = max(0,size-card-x[size:size*2].sum())
    violence[2] = max(0,rho-mu.dot(x[0:size]))
    for i in range(size):
        violence[i+3] = max(0,phi(x[i],x[i+size],t))
        violence[(i+3)+size] = max(0, -x[i+size])
        violence[(i+3)+2*size] = max(0, x[i+size]-1)
        violence[(i+3)+3*size] = max(0, -x[i])
        violence[(i+3)+4*size] = max(0, x[i]-u_i[i])
    viol = max(violence) 
    if (viol <= (1.0/4)*old_viol):
        u1 = u1 + max(-u1, (2*k)*(x.sum()-1))
        u2 = u2 + max(-u2, (2*k)*(rho-mu.dot(x[0:size])))
        u3 = u3 + max(-u3, (2*k)*(size-card-x[size:size*2].sum()))
        for i in range(size):
            u4[i] = u4[i] + max(-u4[i], (2*k)*(x[i]-u_i[i]))
            u5[i] = u5[i] + max(-u5[i], (2*k)*(-x[i]))
            u6[i] = u6[i] + max(-u6[i], (2*k)*(-x[i+size]))
            u7[i] = u7[i] + max(-u7[i], (2*k)*(x[i+size]-1))
            u8[i] = u8[i] + max(-u8[i], (2*k)*phi(x[i], x[i+size], t))
            u9[i] = u9[i] + max(-u9[i], (2*k)*phi_(x[i], x[i+size], t))
    if (viol > (1.0/4)*old_viol):
        t = t*step
        k = (k*(1/step))
    print('{} -> {}'.format(it, x[0:size].dot(sigma).dot(x[0:size])))
    x0 = x
    old_viol = viol

1 -> 3.912326193247455e-12
2 -> 0.6318008102434908
3 -> 0.6128710482710693
4 -> 0.5922306272900774
5 -> 0.5697733388870941
6 -> 0.5454083663818551
7 -> 0.519051347320264
8 -> 0.4906404586295475
9 -> 0.4601418146767629
10 -> 0.42755549713751817
11 -> 0.39293270857843776
12 -> 0.35638795484252683
13 -> 0.31811777652938306
14 -> 0.2784224718246383
15 -> 0.23773187879470684
16 -> 0.19663618858835197
17 -> 0.15592184938953507
18 -> 0.11661305193118966
19 -> 0.08001954577575673
20 -> 0.047790304745493624
21 -> 0.02197346377682528
22 -> 0.005081624038415126
23 -> 6.763570668838901e-10
24 -> 8.383470734719205e-10
25 -> 1.0210016726056713e-09
26 -> 1.2449239685265713e-09
27 -> 1.5197474777646221e-09
28 -> 1.857146720436452e-09
29 -> 2.2713199461679625e-09
30 -> 2.7813175435131373e-09
31 -> 3.4077202274872746e-09
32 -> 4.178657619344687e-09
33 -> 5.127064231915895e-09
34 -> 6.294699487570421e-09
35 -> 7.732410259626282e-09
36 -> 9.503329272625604e-09
37 -> 1.1684731039868903e-08
38 -> 1.43725679

In [110]:
x[0:size].dot(sigma).dot(x[0:size])

71.99746776293883

In [104]:
x[0:size].sum()

0.9858517138167737

In [105]:
viol

1.9835864622452957e-08

In [ ]:
objective(x)

In [ ]:
t